In [4]:
import time
start = time.time()

In [5]:
! pip install mne

In [6]:
from glob import glob # it Is help us to read all files in the Folder 
import os
import mne # It is a Package which is Used to Analysis the EEG Dataset
from numpy import array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
all_file_path = glob('C:/Users/home/TASK_Healthy_and_MDD_fif/*.fif')
print(len(all_file_path))

61


In [8]:
all_file_path

['C:/Users/home/TASK_Healthy_and_MDD_fif\\h10_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h11_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h12_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h13_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h14_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h15_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h16_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h17_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h18_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h19_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h1_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h22_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h23_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MDD_fif\\h24_task_cropped_raw.fif',
 'C:/Users/home/TASK_Healthy_and_MD

In [9]:
healthy_file_path = [i for i in all_file_path if 'h' in i.split('\\')[1]]
patient_file_path = [i for i in all_file_path if 'm' in i.split('\\')[1]]
print(len(healthy_file_path), len(patient_file_path))

28 33


In [10]:
def read_data(file_path):
    data = mne.io.read_raw_fif(file_path, preload = True)
    data.set_eeg_reference()
    data.filter(l_freq = 0.5, h_freq = 60)
    epochs = mne.make_fixed_length_epochs(data, duration = 5, overlap = 2)
    array = epochs.get_data()
    return array

In [11]:
sample_data = read_data(healthy_file_path[0])

Opening raw data file C:/Users/home/TASK_Healthy_and_MDD_fif\h10_task_cropped_raw.fif...
Isotrak not found
    Range : 0 ... 154879 =      0.000 ...   604.996 secs
Ready.
Reading 0 ... 154879  =      0.000 ...   604.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 1691 samples (6.605 sec)

Not setting metadata
201 matching events found
No baseline correction applied
0 projection item

In [12]:
sample_data.shape # no. of epochs, channels, length of signals 

(201, 20, 1280)

In [13]:
%%capture
control_epochs_array = [read_data(i) for i in healthy_file_path]
patient_epochs_array = [read_data(i) for i in patient_file_path]

In [14]:
control_epochs_array[0].shape, control_epochs_array[1].shape, control_epochs_array[2].shape, control_epochs_array[3].shape, control_epochs_array[4].shape

((201, 20, 1280),
 (201, 20, 1280),
 (201, 20, 1280),
 (202, 20, 1280),
 (201, 20, 1280))

In [15]:
patient_epochs_array[0].shape, patient_epochs_array[1].shape, patient_epochs_array[2].shape, patient_epochs_array[3].shape

((228, 20, 1280), (211, 20, 1280), (215, 20, 1280), (209, 20, 1280))

In [16]:
control_epochs_labels = [len(i)*[0] for i in control_epochs_array]
patient_epochs_labels = [len(i)*[1] for i in patient_epochs_array]
len(control_epochs_labels), len(patient_epochs_labels)

(28, 33)

In [17]:
data_list = control_epochs_array + patient_epochs_array
label_list = control_epochs_labels + patient_epochs_labels

In [18]:
epochs_array=control_epochs_array+patient_epochs_array
epochs_labels=control_epochs_labels+patient_epochs_labels
print(len(epochs_array),len(epochs_labels))

61 61


In [19]:
groups = [[i]*len(j) for i,j in enumerate(data_list)]
len(groups)

61

In [20]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
groups_array = np.hstack(groups)
print(data_array.shape, label_array.shape, groups_array.shape)

(12644, 20, 1280) (12644,) (12644,)


In [21]:
from scipy import stats

def mean(x):
    return np.mean(x,axis=-1)
def std(x):  
    return np.std(x,axis=-1)
def ptp(x):
    return np.ptp(x,axis=-1)
def var(x):
    return np.var(x,axis=-1)
def minim(x):
    return np.min(x,axis=-1)
def maxim(x):
    return np.max(x,axis=-1)
def argminim(x):
    return np.argmin(x,axis=-1) 
def argmaxim(x):
    return np.argmax(x,axis=-1)   
                   
def abs_diff_signals(x):
    return np.sum(np.abs(np.diff(x,axis=-1)),axis=-1)
def skewness(x):
    return stats.skew(x,axis=-1)
def kurtosis(x):
    return stats.kurtosis(x,axis=-1)
#def rms(x):
#    return np.sqrt(np.mean(x**2,axis=-1)
def concatenate_features(x):
    return np.concatenate((mean(x),std(x),ptp(x),var(x),minim(x),maxim(x),argminim(x),argmaxim(x),abs_diff_signals(x),\
                           skewness(x),kurtosis(x)),axis=-1) 

In [22]:
features=[]
for d in data_array:
    features.append(concatenate_features(d))

In [23]:
features_array=np.array(features)
features_array.shape

(12644, 220)

In [24]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV

In [25]:
clf=SVC()
gkf=GroupKFold(5)
pipe=Pipeline([('scalar',StandardScaler()),('clf',clf)])
param_grid={'clf__C':[0.1,0.3,0.5,0.7,1,3,5,7]}
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=12)
gscv.fit(features_array,label_array,groups=groups_array)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scalar', StandardScaler()),
                                       ('clf', SVC())]),
             n_jobs=12,
             param_grid={'clf__C': [0.1, 0.3, 0.5, 0.7, 1, 3, 5, 7]})

In [26]:
gscv.best_score_

0.8766658076905827

In [27]:
end = time.time()
print("Total time in sec:",end - start)

Total time in sec: 1388.1756649017334
